In [10]:
import os

In [11]:
%pwd

'd:\\projects\\chest-cancer-classification-using-MLFlow-and-DVC\\research'

In [12]:
os.chdir("../")

In [13]:
%pwd

'd:\\projects\\chest-cancer-classification-using-MLFlow-and-DVC'

In [14]:
# entity is a return type of a function
# dataclass is use for define variable without using self
# forzen = true is use for, not use any other functionality apart from this.

from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [15]:
from cnnClassificationProject.constance import *
from cnnClassificationProject.utils.common import read_yaml, create_directories

In [16]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath= CONFIG_FILE_PATH,
        params_filepath= PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config= DataIngestionConfig(
            root_dir= config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config


In [17]:
import os
import zipfile
import gdown
from cnnClassificationProject import logger
from cnnClassificationProject.utils.common import get_size

In [18]:
class DataIngestion:
    def __init__(self,config: DataIngestionConfig):
        self.config = config

    def download_file(self) -> str:

        """ Fetch Data from the URL
        """
        try:
            dataset_url= self.config.source_URL
            zip_download_dir= self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(f"downloading data from {dataset_url} into file {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]
            prefix = "https://drive.google.com/uc?/export=download&id="
            gdown.download(prefix+file_id,zip_download_dir)

            logger.info(f"downloaded data from {dataset_url} into file {zip_download_dir}")

        except Exception as e:
            raise e
        
    def extract_zip_file(self):
        """
        zip file path : str
        Extract the zip file into Data Directory
        Function returns None
        """

        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file,'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [19]:
try:
    config= ConfigurationManager()
    data_ingestion_config= config.get_data_ingestion_config()
    data_ingestion= DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e


[2024-09-09 19:47:08,133:INFO:common:yaml file: config\config.yaml loaded successfully]
[2024-09-09 19:47:08,145:INFO:common:yaml file: params.yaml loaded successfully]
[2024-09-09 19:47:08,147:INFO:common:created directory at: artifacts]
[2024-09-09 19:47:08,148:INFO:common:created directory at: artifacts/data_ingestion]
[2024-09-09 19:47:08,150:INFO:472352388:downloading data from https://drive.google.com/file/d/1RL46nbo7fMSEiIJ8SBdBduhrExmVOzG1/view?usp=sharing into file artifacts/data_ingestion/data.zip]


Downloading...
From (original): https://drive.google.com/uc?/export=download&id=1RL46nbo7fMSEiIJ8SBdBduhrExmVOzG1
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=1RL46nbo7fMSEiIJ8SBdBduhrExmVOzG1&confirm=t&uuid=3bdcf5ad-2281-4fa2-9700-043816a418f9
To: d:\projects\chest-cancer-classification-using-MLFlow-and-DVC\artifacts\data_ingestion\data.zip
100%|██████████| 49.0M/49.0M [00:07<00:00, 6.37MB/s]

[2024-09-09 19:47:22,178:INFO:472352388:downloaded data from https://drive.google.com/file/d/1RL46nbo7fMSEiIJ8SBdBduhrExmVOzG1/view?usp=sharing into file artifacts/data_ingestion/data.zip]
